In [16]:
# 规范化变量统计

In [16]:
# 导入程序包
import pandas as pd
import numpy as np

import re
import sys
import os
project_path = os.getcwd()

# 变量统计

In [19]:
df_model=pd.read_excel(project_path+'/data/df_修改剂量_model_data_importance_stat.xlsx')
if 'Unnamed: 0' in df_model.columns:
    df_model = df_model.drop(['Unnamed: 0'], axis=1)

In [20]:
df_model.shape

(121, 41)

In [21]:
df_model.columns

Index(['target_日剂量', '体重', '钙离子阻抗剂', '尿酸', '日剂量', '肌酐', '乳酸脱氢酶', '总胆红素', 'age',
       'BMI', '身高', '天门冬氨酸氨基转移酶', '甘油三酯', '糖尿病', '中性粒细胞总数', '直接胆红素', '心型肌酸激酶',
       '凝血酶原活动度', '胱抑素-C', '平均红细胞血红蛋白量', '尿素', '血小板分布宽度', '血红蛋白测定',
       '高密度脂蛋白胆固醇', '酸碱浓度', '肌酸激酶', '低密度脂蛋白胆固醇', '总胆固醇', '中性粒细胞百分数',
       'a-羟丁酸脱氢酶', '淋巴细胞百分数', '平均RBC血红蛋白浓度', '嗜碱性粒细胞百分数', 'test_result',
       '大型血小板比值', '平均血小板容积', '高血压', '糖皮质激素', 'gender', '其他免疫抑制剂', '质子泵抑制剂'],
      dtype='object')

In [49]:
# 选取变量
discrete_col = ['target_日剂量','钙离子阻抗剂','糖尿病','高血压','糖皮质激素','gender','其他免疫抑制剂','质子泵抑制剂']
continuous_col=[x for x in df_model.columns if x not in discrete_col]

## 分类变量统计

In [50]:
# 求分类变量比例
df_discrete_stat=pd.DataFrame(columns=['变量名称','所有事件(%d)' % df_model.shape[0],'缺失率(%)'])
for i in discrete_col:
    print(i)
    # 缺失率
    if df_model[i].isnull().sum()==0:
        miss_rate='0%'
    else:
        miss_rate=df_model[i].isnull().sum()/df_model.shape[0]
        miss_rate="%.2f%%" % (miss_rate * 100)      # 百分数输出
    df_discrete_stat.loc[df_discrete_stat.shape[0],['变量名称','缺失率(%)']]=[i+'，n(%)',miss_rate]
    
    # 分类变量单独统计
    name_list=[]
    num_perc_list=[]
    df_model_stat=df_model[df_model[i].notnull()].sort_values([i],ascending=True)
    
    # 二分类还是多分类变量
    if df_model_stat[i].nunique() <=2:
        if re.match('gender|性别',i):
            name_list=['男','女']
        else:
            name_list=['是','否']
        for name,value in zip(name_list,[1,0]):
            print(name)
            num=df_model_stat[df_model_stat[i]==value].shape[0]
            percent=num/df_model.shape[0]
            percent="%.2f%%" % (percent * 100)
            num_percent=str(num)+'('+percent+')'
            num_perc_list.append(num_percent)
    else:
        for value in sorted(df_model_stat[i].unique()):
            print(value)
            name_list.append(value)
            num=df_model_stat[df_model_stat[i]==value].shape[0]
            percent=num/df_model.shape[0]
            percent="%.2f%%" % (percent * 100)
            num_percent=str(num)+'('+percent+')'
            num_perc_list.append(num_percent)
        
        
    df_temp = pd.DataFrame(data={'变量名称':name_list,
                                 '所有事件(%d)' % df_model.shape[0]:num_perc_list})

    df_discrete_stat=pd.concat([df_discrete_stat,df_temp],axis=0)
    df_discrete_stat=df_discrete_stat.reset_index(drop=True)

target_日剂量
1.0
1.5
2.0
2.5
3.0
4.0
钙离子阻抗剂
是
否
糖尿病
是
否
高血压
是
否
糖皮质激素
是
否
gender
男
女
其他免疫抑制剂
是
否
质子泵抑制剂
是
否


In [51]:
name_list

['是', '否']

In [52]:
# 保存分类变量统计
df_discrete_stat.to_excel(project_path+'/data/stat/df_2.1_分类变量统计_importance.xlsx')

## 连续变量统计

In [53]:
# 统计全变量体系各变量的平均数、上下四分位数、缺失率
feature_quarter_list=[]
mean_quarter_list=[]
feature_std_list=[]
mean_std_list=[]
miss_list=[]
for i in continuous_col:
    # 计算上下四分位、均值、标准差
    try:
        data = df_model[i].astype('float')
        stat_result = pd.DataFrame(data.describe())
        mean_value=stat_result.loc['mean',i]
        up_quarter=stat_result.loc['25%',i]
        down_quarter=stat_result.loc['75%',i]
        std_value=stat_result.loc['std',i]
    except:
        mean_value=np.nan
        up_quarter=np.nan
        down_quarter=np.nan
    # 计算缺失率
    if df_model[i].isnull().sum()==0:
        miss_rate='0%'
    else:
        miss_rate=df_model[i].isnull().sum()/df_model.shape[0]
        miss_rate="%.2f%%" % (miss_rate * 100)      # 百分数输出
    miss_list.append(miss_rate)
    # mean(quarter)
    feature_quarter_list.append(i+'，mean（IQR）')
    mean_quarter_list.append('%.2f(%.2f-%.2f)' % (mean_value,up_quarter,down_quarter))
    # mean(std)
    feature_std_list.append(i+'，mean±std')
    mean_std_list.append('%.2f±%.2f' % (mean_value,std_value))

In [54]:
df_continuous_quarter=pd.DataFrame(data={'特征':feature_quarter_list,
                        'mean_quarter_list':mean_quarter_list,
                        'miss_list':miss_list})
df_continuous_std=pd.DataFrame(data={'特征':feature_std_list,
                        'mean_std_list':mean_std_list,
                        'miss_list':miss_list})

In [55]:
df_continous_stat=pd.concat([df_continuous_quarter,df_continuous_std],axis=1)
# 保存连续变量统计
df_continous_stat.to_excel(project_path+'/data/stat/df_2.2_连续变量统计_importance.xlsx')